https://www.kaggle.com/dredlaw/exploring-product-weekly-sale-quantities

 - To środowisko Python 3 zawiera wiele pomocnych bibliotek analitycznych zainstalowanych
 - Jest zdefiniowany przez obraz dokera kaggle/python: https://github.com/kaggle/docker-python
 - Na przykład, oto kilka pomocnych pakietów do załadowania

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

 - Wejściowe pliki danych są dostępne w katalogu "../input/".
 - Na przykład uruchomienie tego (klikając uruchom lub wciskając Shift+Enter) spowoduje wyświetlenie listy plików w katalogu wejściowym

In [ ]:

#import os
#print(os.listdir("../input"))

# 

Wszelkie wyniki, które zapisujesz w bieżącym katalogu, są zapisywane jako dane wyjściowe.

In [3]:
# Control parameters
autocorr_nlags_max = 13 # Maksymalna liczba opóźnień do obliczenia autokorelacji
    # Użyj 13, ponieważ 13 tygodni to w przybliżeniu jeden kwartał, tj. spójrz na powtarzające się zakupy w ciągu kwartału
corr_thresh = 0.7 # Próg dla wyniku korelacji
    # Use >= 0.7 dla silnie skorelowanych
param_top_perc = 0.05 # Procent produktów o największej średniej tygodniowej ilości sprzedaży do zachowania do dalszej analizy
    # Use 0.05 utrzymać top 40

In [17]:
# Get column names
col_names = pd.read_csv("/media/wojciech/D6DE33C1DE339927/1A/Sales_Transactions_Dataset_Weekly.csv", nrows=0).columns
col_names

Index(['Product_Code', 'W0', 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8',
       ...
       'Normalized 42', 'Normalized 43', 'Normalized 44', 'Normalized 45',
       'Normalized 46', 'Normalized 47', 'Normalized 48', 'Normalized 49',
       'Normalized 50', 'Normalized 51'],
      dtype='object', length=107)

 - Załaduj surowe dane
 - czyli kolumny product_code i tygodniowej kwoty sprzedaży
 - raw_data = pd.read_csv('../input/Sales_Transactions_Dataset_Weekly.csv').filter(regex='Product_Code|^W[0-9]+$')

In [18]:
raw_data = pd.read_csv("/media/wojciech/D6DE33C1DE339927/1A/Sales_Transactions_Dataset_Weekly.csv", 
                       usecols=[col for col in col_names if 'Product_Code' in col or 'W' in col],
                       index_col = 'Product_Code')

# Check shape of data
raw_data.shape
raw_data.head(5)

,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
Product_Code,,,,,,,,,,,,,,,,,,,,,
P1,11,12,10,8,13,12,14,21,6,14,...,4,7,8,10,12,3,7,6,5,10
P2,7,6,3,2,7,1,6,3,3,3,...,2,4,5,1,1,4,5,1,6,0
P3,7,11,8,9,10,8,7,13,12,6,...,6,14,5,5,7,8,14,8,8,7
P4,12,8,13,5,9,6,9,13,13,11,...,9,10,3,4,6,8,14,8,7,8
P5,8,5,13,11,6,7,9,14,9,9,...,7,11,7,12,6,6,5,11,8,9


In [8]:
# Calculate correlation between sale quantities
# Use Spearman method to capture non-linear correlation
product_corr = raw_data.transpose().corr(method='spearman')
print(product_corr.shape)
product_corr.head(5)

(811, 811)


Product_Code,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P810,P811,P812,P813,P814,P815,P816,P817,P818,P819
Product_Code,,,,,,,,,,,,,,,,,,,,,
P1,1.000000,0.105156,-0.033156,-0.060961,0.024067,-0.115742,0.034288,-0.083970,0.254553,0.178971,...,0.010585,0.141629,0.145374,-0.089562,-0.111400,0.132112,-0.193239,0.021788,-0.246475,-0.009766
P2,0.105156,1.000000,0.051913,0.139827,-0.098849,0.139774,0.023473,0.192785,0.054797,-0.061026,...,0.118540,-0.086185,-0.032924,-0.098239,-0.116745,0.011166,-0.048712,-0.114477,-0.115255,0.098174
P3,-0.033156,0.051913,1.000000,0.339249,-0.003896,-0.078649,-0.018139,-0.226969,-0.023219,0.073690,...,-0.324464,-0.033638,-0.240631,0.162646,-0.045399,-0.225324,-0.080792,-0.246736,0.037838,0.120155
P4,-0.060961,0.139827,0.339249,1.000000,0.030796,0.071470,0.064682,-0.130114,-0.117943,-0.154483,...,0.050968,-0.311405,-0.127180,0.280844,-0.005571,0.029330,0.131983,0.040744,-0.254841,0.245973
P5,0.024067,-0.098849,-0.003896,0.030796,1.000000,-0.096208,0.143222,0.020834,0.100637,0.316596,...,-0.047638,-0.006176,-0.032229,0.115919,0.284669,0.095391,-0.087876,0.108401,0.013865,-0.263292


# Znajdź i zachowaj tylko wysokie korelacje

In [20]:
high_corr_set = set() # zapisz poszczególne zaangażowane wartości Product_Code
high_corr_pairs = [] # zapisz pary Product_Code i wartość korelacji

for row in product_corr:
    for col in product_corr.columns:
        if row < col:
            # Uzyskaj wartość korelacji, unikając robienia tego dwa razy
            corr_val = product_corr.loc[row, col]

            if abs(corr_val) >= corr_thresh:
                # Jeśli wartość korelacji przekracza próg, zapisz ją
                #print(row)
                #print(col)
                #print(corr_val)
                high_corr_pairs.append((row, col, corr_val))
                high_corr_set.add(row)
                high_corr_set.add(col)
            
# print(high_corr_list)
# Convert list to a dataframe
high_corr_df = pd.DataFrame(high_corr_pairs, columns=['Product_1', 'Product_2', 'Corr'])
print(f"Number of product pairs with correlation >= {corr_thresh} or <= -{corr_thresh}: {high_corr_df.shape[0]}")
print(f"Number of distinct Product_Code values involved: {len(high_corr_set)}")
high_corr_df.head(5)


Number of product pairs with correlation >= 0.7 or <= -0.7: 14
Number of distinct Product_Code values involved: 20


,Product_1,Product_2,Corr
0,P18,P191,0.769559
1,P215,P279,0.700140
2,P215,P656,0.700140
3,P251,P253,0.700140
4,P251,P468,0.700140


In [10]:
# Spójrz na większość i najmniej popularnych produktów
num_products = raw_data.shape[0]
num_top = math.floor(param_top_perc * num_products)
print(f'Find the most popular and least popular {num_top} products by average weekly sale quantity: ')
print(' ')

# Oblicz średnią tygodniową ilość sprzedaży
avg_weekly_quantity = raw_data.mean(axis=1)

# Find most popular and least popular products
most_pop_products = avg_weekly_quantity.nlargest(n=num_top).index.values.tolist()
least_pop_products = avg_weekly_quantity.nsmallest(n=num_top).index.values.tolist()

# Extract raw data for these products
most_pop_raw_data = raw_data[raw_data.index.isin(most_pop_products)]
least_pop_raw_data = raw_data[raw_data.index.isin(least_pop_products)]

print(f"Have dataframe with {num_top} most popular products: {most_pop_raw_data.shape[0]}")
most_pop_raw_data.head(2)


Find the most popular and least popular 40 products by average weekly sale quantity: 
 
Have dataframe with 40 most popular products: 40


,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
Product_Code,,,,,,,,,,,,,,,,,,,,,
P15,19,45,47,42,29,44,43,36,25,52,...,38,35,33,30,37,30,30,34,38,20
P16,30,27,27,43,29,32,49,41,49,38,...,40,30,33,39,42,45,31,31,23,22


In [11]:
print(f"Have dataframe with {num_top} least popular products: {least_pop_raw_data.shape[0]}")
least_pop_raw_data.head(2)

Have dataframe with 40 least popular products: 40


,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
Product_Code,,,,,,,,,,,,,,,,,,,,,
P215,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


- Sprawdź, czy najpopularniejsze lub najmniej popularne produkty mają silną korelację z jakimkolwiek innym produktem
- Konwertuj zestaw o wysokiej korelacji na listę

In [12]:

high_corr_list = list(high_corr_set)

# Apply the list - most popular
most_pop_with_strong_corr = most_pop_raw_data[most_pop_raw_data.index.isin(high_corr_list)]
print(f"Number of most popular Product_Code values in the high correlation list: {most_pop_with_strong_corr.shape[0]}")
most_pop_with_strong_corr.head(5)

Number of most popular Product_Code values in the high correlation list: 1


,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
Product_Code,,,,,,,,,,,,,,,,,,,,,
P191,48,36,42,35,44,39,37,45,42,40,...,33,20,25,36,36,28,35,35,34,24


- Najmniej popularne produkty są kupowane bardzo rzadko (kilka tygodni bez sprzedaży) i mają ze sobą wysoką korelację
- Ponieważ produkty mają tylko tygodniowe numery sprzedaży, analiza produktów o niskiej popularności kończy się tutaj

In [13]:
# Apply the list - least popular
least_pop_with_strong_corr = least_pop_raw_data[least_pop_raw_data.index.isin(high_corr_list)]
print(f"Number of least popular Product_Code values in the high correlation list: {least_pop_with_strong_corr.shape[0]}")
least_pop_with_strong_corr.head(15)

Number of least popular Product_Code values in the high correlation list: 15


,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
Product_Code,,,,,,,,,,,,,,,,,,,,,
P215,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
P253,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
P279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P351,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
P383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
P466,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
P468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


 -  Zbadaj cykle zakupu najpopularniejszych produktów

 -  Transponuj most_pop_raw_data, aby produkty były wymienione poziomo (tj. w nazwach kolumn)

In [14]:

most_pop_raw_data_horz = most_pop_raw_data.transpose()

# Define function for easy to repeat autocorrelation
def acf(df, nlags_max):
    output_list = []
    for col in df:
        # For each column in the dataframe, extract the column as a series
        series = df[col]
        for nlag in range(1, nlags_max + 1):
            # Calculate autocorrelation for lags from 1 to nlgas_max
            output_list.append((col, nlag, series.autocorr(lag=nlag)))
            
    output_df = pd.DataFrame(output_list, columns=['Product_Code', 'nlag', 'Autocorr'])
    return output_df

# Use the function
most_pop_autocorr = acf(most_pop_raw_data_horz, autocorr_nlags_max)

print(f"Shape of most_pop_autocorr is {most_pop_autocorr.shape}")
most_pop_autocorr.head(autocorr_nlags_max)

Shape of most_pop_autocorr is (520, 3)


,Product_Code,nlag,Autocorr
0,P15,1,0.250955
1,P15,2,0.209691
2,P15,3,0.264781
3,P15,4,0.386127
4,P15,5,0.070833
5,P15,6,0.170801
6,P15,7,0.181742
7,P15,8,0.347373
8,P15,9,-0.084029
9,P15,10,0.138069


- Filtruj wyniki autokorelacji

- Użyj 95% przedziału ufności do identyfikacji statystycznie istotnych autokorelacji

In [16]:

acf_ci = 1.96 / math.sqrt(most_pop_raw_data.shape[0])

# Apply filter
sig_most_pop_autocorr = most_pop_autocorr.loc[abs(most_pop_autocorr['Autocorr']) > acf_ci]

print(f"Significance threshold is {acf_ci}")
print(f"Number of siginicant autocorrelations found: {sig_most_pop_autocorr.shape[0]}")
sig_most_pop_autocorr.head(20)

Significance threshold is 0.3099032106965012
Number of siginicant autocorrelations found: 61


,Product_Code,nlag,Autocorr
3,P15,4,0.386127
7,P15,8,0.347373
13,P16,1,0.377623
28,P24,3,0.352805
38,P24,13,-0.338871
54,P34,3,0.321690
61,P34,10,0.400815
80,P36,3,0.391355
84,P36,7,0.342850
92,P37,2,0.470894
